In [7]:
### load modules

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'


from qa_utils import *

import warnings
warnings.filterwarnings('ignore')

# reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Merian data repo
root = '/projects/MERIAN/repo'
butler = dafButler.Butler(root)

In [9]:
import lsst.daf.butler as dafButler
import scipy.stats as stats
from astropy.io import ascii
from astropy.table import Table
import numpy as np

 


def mad_sigmaclip(array, low=3., high=3.):
    '''
    similar to sigmaclip, but using median and MAD instead of mean and std
    ''' 
    # mask=0 means the value should be discarded
    
    # clip nan
    mask = np.ones(array.shape,dtype=int)
    mask[np.isnan(array)] = 0
    c = array[mask==1]
    
    delta = 1
    while delta:
        c_std = stats.median_abs_deviation(array)
        c_median =  np.nanmedian(c)
        size = c.size
        critlower = c_median - c_std * low
        critupper = c_median + c_std * high
        mask[(array <= critlower) | (array >= critupper)] = 0
        c = c[(c >= critlower) & (c <= critupper)]
        delta = size - c.size

    return mask, array[mask==1]


# random tract
list_tract_ids = [8280,8525,8764,8769,9010,9078,9088,9099,9102,9131,9135,9227,9313,9319,9344,9378,9456,9459,9470,9556,9564,9574,9589,9798,9800,9812,9828,9939,9946,9953,10040,10048,10061,10183,10293,10427]  

collection = 'DECam/runs/merian/dr1_wide'
skymap = 'hsc_rings_v1'

flux_colnames = ['N708_gaap1p0Flux', 'N540_gaap1p0Flux', 'N708_gaap1p5Flux','N540_gaap1p5Flux']

tract_skyobj_dict = {}
for flux_colname in flux_colnames: 
    tract_skyobj_dict[flux_colname] = {}

    tract_skyobj_dict[flux_colname]['tract_id_list'] = []
    tract_skyobj_dict[flux_colname]['tract_median_list'] = []
    tract_skyobj_dict[flux_colname]['tract_std_list'] = []

for i,tract in enumerate(list_tract_ids):

    ### load object table
    try:
        cat_coadd = butler.get( 'objectTable_tract', tract=tract, instrument='DECam',
                        skymap=skymap, collections=collection)
        
        print(f'Has ObjectTable_tract for {tract}')
    except Exception as e:
        print(e)
        print(f'No objectTable_tract for {tract}')
        continue

    ### load sky objects
    try:
        skyobj_cat = cat_coadd[cat_coadd.sky_object==True]
    except Exception as e:
        print(e)
        print(f'No sky objects for {tract}')
        continue

    for flux_colname in flux_colnames:

        print(f'### Working on Tract {tract} {flux_colname} ###')

 
        ### target flux
        try:
            flux = skyobj_cat[flux_colname]
        except Exception as e:
            print(e)
            print(f'No {flux_colname} for {tract}')
            continue
        

        ### Tract Level
        mask, flux_clipped = mad_sigmaclip(flux,low=3,high=3)
        Neff = np.sum(mask)
        
        
        median = np.nanmedian(flux_clipped)
        std = stats.median_abs_deviation(flux_clipped)

        tract_skyobj_dict[flux_colname]['tract_id_list'].append(tract)
        tract_skyobj_dict[flux_colname]['tract_median_list'].append(median)
        tract_skyobj_dict[flux_colname]['tract_std_list'].append(std)
 

        ### Patch Level
        if i==1:
            patch_list = np.unique(skyobj_cat['patch'])
            patch_id_list = []
            patch_median_list = []
            patch_std_list = []

            for patch_id in patch_list:
                patch_cat = skyobj_cat[skyobj_cat['patch']==patch_id]
                patch_flux = patch_cat[flux_colname]
                mask, patch_flux_clipped = mad_sigmaclip(patch_flux)
                Neff = len(patch_flux)
                
                mid_flux = np.nanmedian(patch_flux_clipped)
                std_flux = stats.median_abs_deviation(patch_flux_clipped)

                if np.isnan(mid_flux):
                    print(f'Nan median for patch {patch_id} of tract {tract}')
                    print('Not Nan values:',np.sum(~np.isnan(patch_flux_clipped)))
                    continue
                
                patch_id_list.append(patch_id)
                patch_median_list.append(mid_flux)
                patch_std_list.append(std_flux)

            
            patch_stat = Table([patch_id_list,patch_median_list,patch_std_list],names=['patch','median','std'])
            patch_stat['patch'].description = 'patch id'
            patch_stat['median'].description = f'median of {flux_colname} flux for each patch'
            patch_stat['std'].description = f'std of {flux_colname} flux for each patch'
            patch_stat.meta['tract'] = tract

            patch_stat.meta['comments'] = f'statisitcs of {flux_colname} flux for each patch in tract {tract}'

            ascii.write(patch_stat, f'patch_stat_{tract}_{flux_colname}.ecsv', overwrite=True, format='ecsv')
        
        del flux
        del mask
        del flux_clipped

    del cat_coadd
    del skyobj_cat



for flux_colname in flux_colnames:
    tract_id_list = tract_skyobj_dict[flux_colname]['tract_id_list']
    tract_median_list = tract_skyobj_dict[flux_colname]['tract_median_list']
    tract_std_list = tract_skyobj_dict[flux_colname]['tract_std_list']

    tract_stat = Table([tract_id_list,tract_median_list,tract_std_list],names=['tract','median','std'])
    tract_stat['tract'].description = 'tract id'
    tract_stat['median'].description = f'median of {flux_colname} flux for each tract'
    tract_stat['std'].description = f'std of {flux_colname} flux for each tract'

    tract_stat.meta['comments'] = f'statisitcs of {flux_colname} flux for each tract'

    ascii.write(tract_stat, f'tract_stat_{flux_colname}.ecsv', overwrite=True, format='ecsv')



Has ObjectTable_tract for 8280
### Working on Tract 8280 N708_gaap1p0Flux ###
### Working on Tract 8280 N540_gaap1p0Flux ###
### Working on Tract 8280 N708_gaap1p5Flux ###
### Working on Tract 8280 N540_gaap1p5Flux ###
Has ObjectTable_tract for 8525
### Working on Tract 8525 N708_gaap1p0Flux ###
### Working on Tract 8525 N540_gaap1p0Flux ###
### Working on Tract 8525 N708_gaap1p5Flux ###
### Working on Tract 8525 N540_gaap1p5Flux ###
Has ObjectTable_tract for 8764
### Working on Tract 8764 N708_gaap1p0Flux ###
### Working on Tract 8764 N540_gaap1p0Flux ###
### Working on Tract 8764 N708_gaap1p5Flux ###
### Working on Tract 8764 N540_gaap1p5Flux ###
Dataset objectTable_tract with data ID {skymap: 'hsc_rings_v1', tract: 8769} could not be found in collections DECam/runs/merian/dr1_wide.
No objectTable_tract for 8769
Has ObjectTable_tract for 9010
### Working on Tract 9010 N708_gaap1p0Flux ###
### Working on Tract 9010 N540_gaap1p0Flux ###
### Working on Tract 9010 N708_gaap1p5Flux ###
##